In [ ]:
# === Remove latest 2 episodes: posts, episode folders, GitHub releases, then push ===
import os, re, subprocess, shutil
from pathlib import Path
import requests
import yaml

# Repo roots
ROOT    = Path.cwd()
POSTS   = ROOT / "_posts"
EP_DIR  = ROOT / "episodes"

# Project settings
from chengyu.config import settings
REPO = settings.REPO  # e.g. "kohlenberg/chengyudaily"

# ---- helpers ----
def git(*args):
    print("+ git", " ".join(args))
    subprocess.check_call(["git", *args], cwd=ROOT)

def delete_release_and_tag(repo: str, tag: str):
    """Delete GitHub Release by tag and remove the tag ref. Skips gracefully if not found."""
    token = os.environ.get("GITHUB_TOKEN") or os.environ.get("GH_TOKEN")
    if not token:
        print("⚠️  No GITHUB_TOKEN; skipping release deletion.")
        return
    headers = {"Authorization": f"token {token}", "Accept": "application/vnd.github+json"}
    api = "https://api.github.com"

    # 1) Find release by tag
    url = f"{api}/repos/{repo}/releases/tags/{tag}"
    r = requests.get(url, headers=headers, timeout=30)
    if r.status_code == 200:
        rel = r.json()
        rid = rel["id"]
        print(f"  Found release {rid} for tag {tag}; deleting…")
        dr = requests.delete(f"{api}/repos/{repo}/releases/{rid}", headers=headers, timeout=30)
        if dr.status_code in (204, 200):
            print("  ✔ Release deleted.")
        else:
            print(f"  ⚠️  Failed to delete release: {dr.status_code} {dr.text}")
    elif r.status_code == 404:
        print(f"  (No release for tag {tag})")
    else:
        print(f"  ⚠️  Could not query release by tag: {r.status_code} {r.text}")

    # 2) Delete tag ref (even if release not found)
    ref_url = f"{api}/repos/{repo}/git/refs/tags/{tag}"
    dr = requests.delete(ref_url, headers=headers, timeout=30)
    if dr.status_code in (204, 200):
        print("  ✔ Git tag deleted.")
    elif dr.status_code == 404:
        print("  (No git tag to delete.)")
    else:
        print(f"  ⚠️  Failed to delete tag: {dr.status_code} {dr.text}")

def list_latest_posts(n=2):
    posts = sorted(POSTS.glob("*.md"))
    return posts[-n:] if len(posts) >= n else posts

def post_to_folder(md_path: Path) -> str:
    date = md_path.name[:10]
    slug = md_path.stem[11:]  # after 'YYYY-MM-DD-'
    return f"{date}-{slug}"

def safe_git_rm(path: Path):
    """git rm if tracked; also delete from disk to be safe."""
    rel = str(path.relative_to(ROOT))
    try:
        git("rm", "-rf", "--ignore-unmatch", rel)
    except subprocess.CalledProcessError:
        print(f"  (not tracked in git) {rel}")
    # remove from disk if still present
    if path.is_dir():
        shutil.rmtree(path, ignore_errors=True)
    elif path.exists():
        try: path.unlink()
        except Exception: pass

# ---- main ----
targets = list_latest_posts(2)
if not targets:
    print("No posts found.")
else:
    print("Will remove these posts (latest first):")
    for md in targets:
        print(" -", md.name)

    for md in targets:
        date = md.name[:10]
        slug = md.stem[11:]
        folder = post_to_folder(md)
        tag = f"v{date.replace('-','')}-{slug}"

        print(f"\nRemoving episode: {folder}")
        # 1) Delete GitHub release/tag
        delete_release_and_tag(REPO, tag)

        # 2) Remove files from repo
        ep_path = EP_DIR / folder
        print("  Deleting episode folder:", ep_path)
        safe_git_rm(ep_path)

        print("  Deleting post file:", md)
        safe_git_rm(md)

    # 3) Commit & push
    try:
        git("commit", "-m", f"Remove latest 2 episodes")
    except subprocess.CalledProcessError:
        print("Nothing to commit (maybe files were already gone).")
    else:
        git("push", "origin", "main")
        print("✔ Pushed. Pages/Actions will rebuild the RSS and site.")
